'''
Author:
        
        KIM, JoengYoong, jeongyoong@ccnets.org
        
    COPYRIGHT (c) 2024. CCNets. All Rights reserved.
'''

# Annotated food crops and weed images


https://www.kaggle.com/datasets/sukhmandeepsinghbrar/annotated-food-crops-and-weed-images

In [1]:
import sys

path_append = "../../" # Go up one directory from where you are.
sys.path.append(path_append) 

from tools.setting.ml_params import MLParameters
from tools.setting.data_config import DataConfig
from nn.utils.init import set_random_seed
set_random_seed(0)

In [2]:
dataset_path = '../data/Annotated food crops and weed images/Ronin_OPEN_DB'

In [3]:
import random
from collections import defaultdict
import os
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset
from PIL import Image
import xml.etree.ElementTree as ET

# custom dataset class for the FoodOrWeed dataset
class FoodOrWeedDataset(Dataset):
    def __init__(self, root_dir, split='train', transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.annotations_dir = os.path.join(root_dir, 'annotations')
        self.images_dir = os.path.join(root_dir, 'raw images')
        self.data = []
        self.class_to_idx = {}  
        self.indices_by_class = defaultdict(list) 

        # Load all annotation files and corresponding image paths
        for annotation_file in os.listdir(self.annotations_dir):
            if annotation_file.endswith('.xml'):
                annotation_path = os.path.join(self.annotations_dir, annotation_file)
                image_file = annotation_file.replace('.xml', '.jpg')
                image_path = os.path.join(self.images_dir, image_file)
                self.load_annotation(annotation_path, image_path)

        self.data_split(split)
    # Load annotation file and corresponding image path
    def load_annotation(self, annotation_path, image_path):
        tree = ET.parse(annotation_path)
        root = tree.getroot()
        for obj in root.findall('object'):
            name = obj.find('name').text
            if name not in self.class_to_idx:
                self.class_to_idx[name] = len(self.class_to_idx)
            idx = len(self.data)
            self.indices_by_class[name].append(idx)  
            bndbox = obj.find('bndbox')
            xmin = int(bndbox.find('xmin').text)
            ymin = int(bndbox.find('ymin').text)
            xmax = int(bndbox.find('xmax').text)
            ymax = int(bndbox.find('ymax').text)
            self.data.append((image_path, self.class_to_idx[name], (xmin, ymin, xmax, ymax)))
    # Split data into train, validation and test sets
    def data_split(self, split):
        split_data = []
        for label, indices in self.indices_by_class.items():
            random.shuffle(indices)
            if split == 'train':
                split_data.extend(indices[:int(0.6 * len(indices))])
            elif split == 'val':
                split_data.extend(indices[int(0.6 * len(indices)):int(0.8 * len(indices))])
            elif split == 'test':
                split_data.extend(indices[int(0.8 * len(indices)):])
        self.data = [self.data[i] for i in split_data]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        image_path, label_idx, (xmin, ymin, xmax, ymax) = self.data[index]
        image = Image.open(image_path).convert('RGB')
        image = image.crop((xmin, ymin, xmax, ymax))
        if self.transform:
            image = self.transform(image)
        label = torch.tensor(label_idx, dtype=torch.long)
        label = F.one_hot(label, num_classes=len(self.class_to_idx))
        return image, label


In [4]:
from torchvision import transforms

# Create the datasets and data loaders
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4716, 0.4451, 0.4272], std=[0.1314, 0.1255, 0.1245])
])
train_dataset = FoodOrWeedDataset(path_append + dataset_path, split='train', transform=transform)
val_dataset = FoodOrWeedDataset(path_append + dataset_path, split='val', transform=transform)
test_dataset = FoodOrWeedDataset(path_append + dataset_path, split='test', transform=transform)

# Print the shape of samples in each dataset
X, y = train_dataset[0]
print(X.shape)
print(y.shape)

torch.Size([3, 224, 224])
torch.Size([2])


In [5]:
data_config = DataConfig(dataset_name = 'CropOrWeed', task_type='binary_classification', obs_shape=[3, 224, 224], label_size=2)

#  Set training configuration from the AlgorithmConfig class, returning them as a Namespace object.
ml_params = MLParameters(ccnet_network = 'resnet', encoder_network = 'none')
ml_params.model.ccnet_config.d_model = 256
ml_params.training.num_epoch = 10

In [6]:
from trainer_hub import TrainerHub

# Set the device to GPU if available, else CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

# Initialize the TrainerHub class with the training configuration, data configuration, device, and use_print and use_wandb flags
trainer_hub = TrainerHub(ml_params, data_config, device, use_print=True, use_wandb=False, print_interval=5)

In [7]:
trainer_hub.summary()

=== Trainer Hub Summary ===
Device: cuda
Use Core Network: True
Use Encoder Network: False
Use WANDB Logging: False

--- Core Network Configuration ---
Core Model: resnet
Core Config: ModelConfig(model_name='resnet', num_layers=6, d_model=256, dropout=0.05, obs_shape=[3, 224, 224], condition_dim=2, z_dim=128)

Training Parameters:
Batch Size: 64
Evaluation Batch Size: 256
Test Batch Size: 640
Number of Epochs: 10
Maximum Iterations: 1000000

Algorithm Parameters:
Enable Diffusion: False

Optimization Parameters:
Learning Rate: 0.0002
Decay Rate: 0.01
Scheduler Type: exponential


In [8]:
trainer_hub.train(train_dataset, val_dataset)

Epochs:   0%|          | 0/10 [00:00<?, ?it/s]

Iterations:   0%|          | 0/73 [00:00<?, ?it/s]

[0/10][5/73][Time 7.70]
Unified LR across all optimizers: 0.00019994474559179828
--------------------Training Metrics--------------------
Trainer:  resnet
Inf: 0.2775	Gen: 1.1287	Rec: 1.1596	E: 0.2466	R: 0.3084	P: 2.0107
--------------------Test Metrics------------------------
accuracy: 0.9707
precision: 0.9708
recall: 0.9707
f1_score: 0.9707

[0/10][10/73][Time 6.30]
Unified LR across all optimizers: 0.0001998987119127626
--------------------Training Metrics--------------------
Trainer:  resnet
Inf: 0.0756	Gen: 0.8970	Rec: 0.8915	E: 0.0811	R: 0.0701	P: 1.7129
--------------------Test Metrics------------------------
accuracy: 1.0000
precision: 1.0000
recall: 1.0000
f1_score: 1.0000

[0/10][15/73][Time 6.34]
Unified LR across all optimizers: 0.00019985268883215303
--------------------Training Metrics--------------------
Trainer:  resnet
Inf: 0.0751	Gen: 0.8618	Rec: 0.8599	E: 0.0770	R: 0.0733	P: 1.6465
--------------------Test Metrics------------------------
accuracy: 0.9805
precision: 0

Iterations:   0%|          | 0/73 [00:00<?, ?it/s]

[1/10][2/73][Time 6.24]
Unified LR across all optimizers: 0.0001993012376544385
--------------------Training Metrics--------------------
Trainer:  resnet
Inf: 0.0135	Gen: 0.7903	Rec: 0.7905	E: 0.0133	R: 0.0137	P: 1.5672
--------------------Test Metrics------------------------
accuracy: 0.9570
precision: 0.9571
recall: 0.9570
f1_score: 0.9570

[1/10][7/73][Time 6.26]
Unified LR across all optimizers: 0.00019925535213152345
--------------------Training Metrics--------------------
Trainer:  resnet
Inf: 0.0314	Gen: 0.8422	Rec: 0.8449	E: 0.0287	R: 0.0341	P: 1.6558
--------------------Test Metrics------------------------
accuracy: 0.9746
precision: 0.9748
recall: 0.9746
f1_score: 0.9746

[1/10][12/73][Time 6.25]
Unified LR across all optimizers: 0.00019920947717292416
--------------------Training Metrics--------------------
Trainer:  resnet
Inf: 0.0151	Gen: 0.8412	Rec: 0.8419	E: 0.0144	R: 0.0158	P: 1.6680
--------------------Test Metrics------------------------
accuracy: 0.9961
precision: 0.

...

[7/10][4/73][Time 6.21]
Unified LR across all optimizers: 0.00019530348623487907
--------------------Training Metrics--------------------
Trainer:  resnet
Inf: 0.0113	Gen: 0.5886	Rec: 0.5884	E: 0.0115	R: 0.0110	P: 1.1658
--------------------Test Metrics------------------------
accuracy: 0.8125
precision: 0.8125
recall: 0.8125
f1_score: 0.8125

[7/10][9/73][Time 6.34]
Unified LR across all optimizers: 0.00019525852112227618
--------------------Training Metrics--------------------
Trainer:  resnet
Inf: 0.0104	Gen: 0.5822	Rec: 0.5816	E: 0.0111	R: 0.0098	P: 1.1534
--------------------Test Metrics------------------------
accuracy: 1.0000
precision: 1.0000
recall: 1.0000
f1_score: 1.0000

[7/10][14/73][Time 6.22]
Unified LR across all optimizers: 0.00019521356636208122
--------------------Training Metrics--------------------
Trainer:  resnet
Inf: 0.0117	Gen: 0.5862	Rec: 0.5852	E: 0.0127	R: 0.0107	P: 1.1597
--------------------Test Metrics------------------------
accuracy: 1.0000
precision: 1

Iterations:   0%|          | 0/73 [00:00<?, ?it/s]

[8/10][1/73][Time 6.25]
Unified LR across all optimizers: 0.00019467491586052795
--------------------Training Metrics--------------------
Trainer:  resnet
Inf: 0.0186	Gen: 0.5660	Rec: 0.5652	E: 0.0194	R: 0.0178	P: 1.1126
--------------------Test Metrics------------------------
accuracy: 0.8125
precision: 0.8125
recall: 0.8125
f1_score: 0.8125

[8/10][6/73][Time 6.22]
Unified LR across all optimizers: 0.0001946300954649407
--------------------Training Metrics--------------------
Trainer:  resnet
Inf: 0.0125	Gen: 0.5645	Rec: 0.5638	E: 0.0132	R: 0.0118	P: 1.1159
--------------------Test Metrics------------------------
accuracy: 1.0000
precision: 1.0000
recall: 1.0000
f1_score: 1.0000

[8/10][11/73][Time 6.24]
Unified LR across all optimizers: 0.00019458528538844283
--------------------Training Metrics--------------------
Trainer:  resnet
Inf: 0.0123	Gen: 0.5602	Rec: 0.5597	E: 0.0129	R: 0.0117	P: 1.1076
--------------------Test Metrics------------------------
accuracy: 1.0000
precision: 1.

Iterations:   0%|          | 0/73 [00:00<?, ?it/s]

[9/10][3/73][Time 6.28]
Unified LR across all optimizers: 0.00019400369235087007
--------------------Training Metrics--------------------
Trainer:  resnet
Inf: 0.0079	Gen: 0.5385	Rec: 0.5385	E: 0.0079	R: 0.0078	P: 1.0691
--------------------Test Metrics------------------------
accuracy: 1.0000
precision: 1.0000
recall: 1.0000
f1_score: 1.0000

[9/10][8/73][Time 6.25]
Unified LR across all optimizers: 0.0001939590264924151
--------------------Training Metrics--------------------
Trainer:  resnet
Inf: 0.0167	Gen: 0.5837	Rec: 0.5834	E: 0.0170	R: 0.0165	P: 1.1503
--------------------Test Metrics------------------------
accuracy: 1.0000
precision: 1.0000
recall: 1.0000
f1_score: 1.0000

[9/10][13/73][Time 6.23]
Unified LR across all optimizers: 0.00019391437091747016
--------------------Training Metrics--------------------
Trainer:  resnet
Inf: 0.0111	Gen: 0.5404	Rec: 0.5400	E: 0.0115	R: 0.0107	P: 1.0694
--------------------Test Metrics------------------------
accuracy: 1.0000
precision: 1.

In [11]:
trainer_hub.test(test_dataset)